<a href="https://colab.research.google.com/github/PriyanshuMahor/PriyanshuMahor/blob/main/AI_AGENT_Masterclass2_Hospital_wards_alloting_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [ ]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [ ]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Gemini API Key: ")

Enter your Gemini API Key: ··········


# STEP 2: Initialize Gemini 1.5 Flash

In [ ]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-1.5-flash-latest", temperature=0.5)

# STEP 3: Node to ask user for symptom

In [ ]:
def get_symptom(state: dict) ->dict:
  symptom = input("Priyanshu's Aspatal me Swagat Hai, Apni Bimari Batao")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [ ]:
def classify_symptom(state: dict)->dict:
  prompt ={
      "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
      "-General\n -Emergency \n -mental health \n"
      f"symptom: {state['symptom']} \n"
      "Respond only with one word: General, Emergenecy or Mental Health"
      "Example : input: I have fever, Output: General"
  }
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as : {category}") #debug
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [ ]:
def symptom_router(state: dict)->dict:
  category2 = state["category"].lower()
  if "general" in category2:
    return "general"
  elif "emergency" in category2:
    return "emergency"
  elif "mental health" in category2:
    return "mental health"
  else:
    return "general"

# STEP 6: Category-specific response nodes


In [ ]:
def general_node(state: dict)->dict:
  state["answer"] = f" '{state['symptom']}' : seems general: directing you to general ward for consulting a doctor."
  return state
def emergency_node(state: dict)->dict:
  state["answer"] = f" '{state['symptom']}' : It is a Medical Emergency: seeking immediate help"
  return state

def mental_health_node(state: dict)->dict:
  state["answer"] = f" '{state['symptom']}' : seems mental health related: talk to our counsellor."
  return state

# STEP 7: Build LangGraph


In [ ]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom","classify")
builder.add_conditional_edges("classify", symptom_router,{
    "general" : "general",
    "emergency" : "emergency",
    "mental_health" : "mental_health"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)

# STEP 8: Compile and invoke the graph


In [ ]:
graph = builder.compile()
final_state = graph.invoke({})
print("Final Output \n")
print(final_state["answers"])

Priyanshu's Aspatal me Swagat Hai, Apni Bimari BataoI have high fever


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).